# Common imports & library functions

In [1]:
import collections
from collections import Counter
from dataclasses import dataclass
import doctest
import functools
import itertools
import math
import re
from utils import product

# Day 1: Report Repair

In [59]:
def pick_numbers_that_sum_to(ns, pick_n, target):
    """
    Returns first `pick_n` numbers in `ns` to sum to `target`.
    >>> pick_numbers_that_sum_to([1, 9, 11, 3], 2, 12)
    (1, 11)
    >>> pick_numbers_that_sum_to([1721, 979, 366, 299, 675, 1456], 2, 2020)
    (1721, 299)
    >>> pick_numbers_that_sum_to([1721, 979, 366, 299, 675, 1456], 3, 2020)
    (979, 366, 675)
    """
    ns = [n for n in ns if n < target]
    return next(p for p in itertools.product(*[ns]*pick_n) if sum(p) == target)

In [41]:
doctest.run_docstring_examples(pick_numbers_that_sum_to, globs=None, verbose=True)

Finding tests in NoName
Trying:
    pick_numbers_that_sum_to([1, 9, 11, 3], 2, 12)
Expecting:
    (1, 11)
ok
Trying:
    pick_numbers_that_sum_to([1721, 979, 366, 299, 675, 1456], 2, 2020)
Expecting:
    (1721, 299)
ok
Trying:
    pick_numbers_that_sum_to([1721, 979, 366, 299, 675, 1456], 3, 2020)
Expecting:
    (979, 366, 675)
ok


In [46]:
# Final answers
with open('day1.txt') as f:
    ns = [int(l.strip()) for l in f]
    print('Part 1: ', product(pick_numbers_that_sum_to(ns, 2, 2020)))
    print('Part 2: ', product(pick_numbers_that_sum_to(ns, 3, 2020)))

Part 1:  876459
Part 2:  116168640


In [19]:
len([n for n in ns if n + 35 > 2020])

7

# Day 2: Password Philosophy

In [77]:
from dataclasses import dataclass
import re

line_re = re.compile(r'(\d+)-(\d+) (\w): (\w+)')

def parse_line(line):
    """
    >>> parse_line('1-3 a: abcde')
    ((1, 3, 'a'), 'abcde')
    """
    lo, hi, el, pw = line_re.match(line).groups()
    return ((int(lo), int(hi), el), pw)

def password_conforms_to_rule_original(rule, password):
    """
    >>> password_conforms_to_rule_original(*parse_line('1-3 a: abcde'))
    True
    >>> password_conforms_to_rule_original(*parse_line('1-3 b: cdefg'))
    False
    >>> password_conforms_to_rule_original(*parse_line('2-9 c: ccccccccc'))
    True
    """
    counts = collections.Counter(password)
    lo, hi, el = rule
    return lo <= counts.get(el, 0) <= hi

def password_conforms_to_rule_official(rule, password):
    """
    >>> password_conforms_to_rule_official(*parse_line('1-3 a: abcde'))
    True
    >>> password_conforms_to_rule_official(*parse_line('1-3 b: cdefg'))
    False
    >>> password_conforms_to_rule_official(*parse_line('2-9 c: ccccccccc'))
    False
    """
    lo, hi, el = rule
    return (password[lo-1] == el) ^ (password[hi-1] == el)

In [78]:
doctest.run_docstring_examples(parse_line, globs=None, verbose=True)
doctest.run_docstring_examples(password_conforms_to_rule_original, globs=None, verbose=True)
doctest.run_docstring_examples(password_conforms_to_rule_official, globs=None, verbose=True)

Finding tests in NoName
Trying:
    parse_line('1-3 a: abcde')
Expecting:
    ((1, 3, 'a'), 'abcde')
ok
Finding tests in NoName
Trying:
    password_conforms_to_rule_original(*parse_line('1-3 a: abcde'))
Expecting:
    True
ok
Trying:
    password_conforms_to_rule_original(*parse_line('1-3 b: cdefg'))
Expecting:
    False
ok
Trying:
    password_conforms_to_rule_original(*parse_line('2-9 c: ccccccccc'))
Expecting:
    True
ok
Finding tests in NoName
Trying:
    password_conforms_to_rule_official(*parse_line('1-3 a: abcde'))
Expecting:
    True
ok
Trying:
    password_conforms_to_rule_official(*parse_line('1-3 b: cdefg'))
Expecting:
    False
ok
Trying:
    password_conforms_to_rule_official(*parse_line('2-9 c: ccccccccc'))
Expecting:
    False
ok


In [79]:
# Final answers
with open('day2.txt') as f:
    rules_and_pws = [parse_line(l.strip()) for l in f]
    original_pws = [l for l in rules_and_pws if password_conforms_to_rule_original(*l)]
    official_pws = [l for l in rules_and_pws if password_conforms_to_rule_official(*l)]
    print('Part 1: ', len(original_pws))
    print('Part 2: ', len(official_pws))

Part 1:  445
Part 2:  491


# Day 3: Toboggan Trajectory

In [56]:
from itertools import count

TREE = '#'

@dataclass
class Map:
    map: str
    w: int
    h: int

    def at(self, x, y):
        return self.map[(self.w + 1) * (y % self.h) + (x % self.w)]

def parse_map(map):
    clean_map = map.strip().replace('\n', '|')
    h = clean_map.count('|') + 1
    w = clean_map.index('|')
    return Map(map=clean_map, h=h, w=w)

def num_trees_hit(dx, dy, map):
    hits = 0
    for x, y in zip(count(0, dx), range(0, map.h, dy)):
        if map.at(x, y) == TREE:
            hits += 1
    return hits

In [57]:
test_map = parse_map("""
..##.......
#...#...#..
.#....#..#.
..#.#...#.#
.#...##..#.
..#.##.....
.#.#.#....#
.#........#
#.##...#...
#...##....#
.#..#...#.#
""")
assert num_trees_hit(dx=3, dy=1, map=test_map) == 7, 'test failed'

In [60]:
# Final answers
with open('day3.txt') as f:
    map = parse_map(f.read())
    print('Part 1: ', num_trees_hit(dx=3, dy=1, map=map))
    product_of_trees_hit = product(num_trees_hit(dx, dy, map)
                                   for dx, dy in ((1, 1), (3, 1), (5, 1), (7, 1), (1, 2)))
    print('Part 2: ', product_of_trees_hit)

Part 1:  151
Part 2:  7540141059


# Day 4: Passport Processing

In [147]:
from typing import Optional

def parse_passport(passport):
    r"""
    >>> parse_passport("pid:100")
    {'pid': '100'}
    >>> parse_passport("ecl:gry  pid:860033327 eyr:2020 hcl:#fffffd\nbyr:1937 iyr:2017 cid:147 hgt:183cm")
    {'ecl': 'gry', 'pid': '860033327', 'eyr': '2020', 'hcl': '#fffffd', 'byr': '1937', 'iyr': '2017', 'cid': '147', 'hgt': '183cm'}
    """
    return dict(p.split(':') for p in re.split('\s+', passport) if p)

def validate_fields_part1(byr, iyr, eyr, hgt, hcl, ecl, pid, cid=None):
    return True

def validate_fields_part2(byr, iyr, eyr, hgt, hcl, ecl, pid, cid=None):
    """
    >>> validate_fields_part2(byr='-1990', iyr='2010', eyr='2020', hgt='155cm', hcl='#fffffd', ecl='amb', pid='860033327')
    False
    >>> validate_fields_part2(byr='1980', iyr='2010', eyr='2020', hgt='155cm', hcl='#ffd', ecl='amb', pid='860033327')
    False
    >>> validate_fields_part2(byr='1980', iyr='2010', eyr='2020', hgt='155cm', hcl='#fffffd', ecl='amb', pid='860033327')
    True
    """
    if not (1920 <= int(byr) <= 2002):
        return False
    if not (2010 <= int(iyr) <= 2020):
        return False
    if not (2020 <= int(eyr) <= 2030):
        return False 
    if hgt.endswith('cm'):
        if not (150 <= int(hgt[:-2]) <= 193):
            return False
    elif hgt.endswith('in'):
        if not (59 <= int(hgt[:-2]) <= 76):
            return False
    else:
        return False
    if not re.match('^#[0-9a-f]{6}$', hcl):
        return False
    if ecl not in {'amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'}:
        return False
    if len(pid) != 9 or not int(pid):
        return False
    return True


def is_valid_passport(passport, validator):
    r"""
    >>> is_valid_passport("", validator=validate_fields_part1)
    False
    >>> is_valid_passport("iyr:2019 "
    ...                   "hcl:#602927 eyr:1967 hgt:170cm "
    ...                   "ecl:grn pid:012533040 byr:1946",
    ...                   validator=validate_fields_part1)
    True
    >>> is_valid_passport("ecl:gry pid:860033327 eyr:2020 hcl:#fffffd byr:1937 iyr:2017 hgt:183cm",
    ...                   validator=validate_fields_part1)
    True
    >>> is_valid_passport("ecl:gry pid:860033327 eyr:2020 hcl:#fffffd byr:1900 iyr:2017 hgt:183cm",
    ...                   validator=validate_fields_part2)
    False
    >>> is_valid_passport("iyr:2019 "
    ...                   "hcl:#602927 eyr:1967 hgt:170cm "
    ...                   "ecl:grn pid:012533040 byr:1946",
    ...                   validator=validate_fields_part2)
    False
    """
    try:
        return validator(**parse_passport(passport))
    except:
        return False

def count_valid_passports(ps, validator):
    return sum(1 if is_valid_passport(p, validator) else 0 for p in ps)

In [148]:
doctest.run_docstring_examples(parse_passport, globs=None, verbose=True)
doctest.run_docstring_examples(validate_fields_part2, globs=None, verbose=True)
doctest.run_docstring_examples(is_valid_passport, globs=None, verbose=True)

Finding tests in NoName
Trying:
    parse_passport("pid:100")
Expecting:
    {'pid': '100'}
ok
Trying:
    parse_passport("ecl:gry  pid:860033327 eyr:2020 hcl:#fffffd\nbyr:1937 iyr:2017 cid:147 hgt:183cm")
Expecting:
    {'ecl': 'gry', 'pid': '860033327', 'eyr': '2020', 'hcl': '#fffffd', 'byr': '1937', 'iyr': '2017', 'cid': '147', 'hgt': '183cm'}
ok
Finding tests in NoName
Trying:
    validate_fields_part2(byr='-1990', iyr='2010', eyr='2020', hgt='155cm', hcl='#fffffd', ecl='amb', pid='860033327')
Expecting:
    False
ok
Trying:
    validate_fields_part2(byr='1980', iyr='2010', eyr='2020', hgt='155cm', hcl='#ffd', ecl='amb', pid='860033327')
Expecting:
    False
ok
Trying:
    validate_fields_part2(byr='1980', iyr='2010', eyr='2020', hgt='155cm', hcl='#fffffd', ecl='amb', pid='860033327')
Expecting:
    True
ok
Finding tests in NoName
Trying:
    is_valid_passport("", validator=validate_fields_part1)
Expecting:
    False
ok
Trying:
    is_valid_passport("iyr:2019 "
                    

In [130]:
# Final answers
with open('day4.txt') as f:
    passports = f.read().split('\n\n')
    print('Part 1: ', count_valid_passports(passports, validator=validate_fields_part1))
    print('Part 2: ', count_valid_passports(passports, validator=validate_fields_part2))

Part 1:  254
Part 2:  184


# Day 5: Binary Boarding

In [218]:
_seat_to_id = str.maketrans('FLBR', '0011')
_row_id_to_seat = str.maketrans('01', 'FB')
_col_id_to_seat = str.maketrans('01', 'LR')

def decode_seat_id(seat_code):
    """
    >>> decode_seat_id('BFFFBBFRRR')
    567
    >>> decode_seat_id('FFFBBBFRRR')
    119
    >>> decode_seat_id('BBFFBBFRLL')
    820
    """
    return int(seat_code.translate(_seat_to_id), base=2)

def encode_seat_id(seat_id):
    """
    >>> encode_seat_id(567)
    'BFFFBBFRRR'
    >>> encode_seat_id(119)
    'FFFBBBFRRR'
    >>> encode_seat_id(820)
    'BBFFBBFRLL'
    """
    code = f'{seat_id:010b}'
    row = code[:-3].translate(_row_id_to_seat)
    col = code[-3:].translate(_col_id_to_seat)
    return row + col

def find_missing_seat(seat_codes):
    seat_ids = {decode_seat_id(c) for c in seat_codes}
    for sid in seat_ids:
        if sid + 1 not in seat_ids and sid + 2 in seat_ids:
            return sid + 1, encode_seat_id(sid + 1)

In [219]:
doctest.run_docstring_examples(decode_seat_id, globs=None, verbose=True)
doctest.run_docstring_examples(encode_seat_id, globs=None, verbose=True)

Finding tests in NoName
Trying:
    decode_seat_id('BFFFBBFRRR')
Expecting:
    567
ok
Trying:
    decode_seat_id('FFFBBBFRRR')
Expecting:
    119
ok
Trying:
    decode_seat_id('BBFFBBFRLL')
Expecting:
    820
ok
Finding tests in NoName
Trying:
    encode_seat_id(567)
Expecting:
    'BFFFBBFRRR'
ok
Trying:
    encode_seat_id(119)
Expecting:
    'FFFBBBFRRR'
ok
Trying:
    encode_seat_id(820)
Expecting:
    'BBFFBBFRLL'
ok


In [220]:
# Final answers
with open('day5.txt') as f:
    seat_codes = {l.strip() for l in f}
    print('Part 1: ', max(decode_seat_id(c) for c in seat_codes))
    print('Part 2: ', find_missing_seat(seat_codes))

Part 1:  991
Part 2:  (534, 'BFFFFBFRRL')
